In [1]:
import healpix as hp
import numpy as np
import xarray as xr

import easygems.healpix as egh
import easygems.remap as egr


ds = xr.open_dataset(
    "https://eerie.cloud.dkrz.de/datasets/icon-esm-er.eerie-control-1950.v20240618.atmos.native.2d_6h_inst/kerchunk",
    engine="zarr",
    chunks={},
)
ds


/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


<xarray.Dataset> Size: 8TB
Dimensions:             (ncells: 5242880, height: 1, height_2: 1, time: 73051)
Coordinates:
    cell_sea_land_mask  (ncells) float64 42MB dask.array<chunksize=(5242880,), meta=np.ndarray>
  * height              (height) float64 8B 10.0
  * height_2            (height_2) float64 8B 2.0
    lat                 (ncells) float64 42MB dask.array<chunksize=(5242880,), meta=np.ndarray>
    lon                 (ncells) float64 42MB dask.array<chunksize=(5242880,), meta=np.ndarray>
  * time                (time) datetime64[ns] 584kB 1991-01-01T05:59:59 ... 2...
Dimensions without coordinates: ncells
Data variables:
    hus2m               (time, height_2, ncells) float32 2TB dask.array<chunksize=(1, 1, 5242880), meta=np.ndarray>
    psl                 (time, ncells) float32 2TB dask.array<chunksize=(1, 5242880), meta=np.ndarray>
    ts                  (time, ncells) float32 2TB dask.array<chunksize=(1, 5242880), meta=np.ndarray>
    uas                 (time, height, ncells) float32 2TB dask.array<chunksize=(1, 1, 5242880), meta=np.ndarray>
    vas                 (time, height, ncells) float32 2TB dask.array<chunksize=(1, 1, 5242880), meta=np.ndarray>
Attributes: (12/31)
    Conventions:           CF-1.7 CMIP-6.2
    activity_id:           EERIE
    data_specs_version:    01.00.32
    forcing_index:         1
    initialization_index:  1
    license:               EERIE model data produced by MPI-M is licensed und...
    ...                    ...
    parent_activity_id:    EERIE
    sub_experiment_id:     none
    experiment:            coupled control with fixed 1950's forcing (HighRes...
    source:                ICON-ESM-ER (2023): \naerosol: none, prescribed MA...
    institution:           Max Planck Institute for Meteorology, Hamburg 2014...
    sub_experiment:        none

In [2]:
order = zoom = 9
nside = hp.order2nside(order)
npix = hp.nside2npix(nside)

hp_lon, hp_lat = hp.pix2ang(nside=nside, ipix=np.arange(npix), lonlat=True, nest=True)
hp_lon = (hp_lon + 180) % 360 - 180  # [-180, 180)
hp_lon += 360 / (4 * nside) / 4  # shift quarter-width

In [3]:
weights = egr.compute_weights_delaunay((ds.lon, ds.lat), (hp_lon, hp_lat))

In [4]:
ds_remap = xr.apply_ufunc(
    egr.apply_weights,
    ds,
    kwargs=weights,
    keep_attrs=True,
    input_core_dims=[["ncells"]],
    output_core_dims=[["cell"]],
    output_dtypes=["f4"],
    vectorize=True,
    dask="parallelized",
    dask_gufunc_kwargs={
        "output_sizes": {"cell": npix},
    },
)
ds_remap

<xarray.Dataset> Size: 5TB
Dimensions:   (time: 73051, height_2: 1, cell: 3145728, height: 1)
Coordinates:
  * height    (height) float64 8B 10.0
  * height_2  (height_2) float64 8B 2.0
  * time      (time) datetime64[ns] 584kB 1991-01-01T05:59:59 ... 2040-12-31T...
Dimensions without coordinates: cell
Data variables:
    hus2m     (time, height_2, cell) float32 919GB dask.array<chunksize=(1, 1, 3145728), meta=np.ndarray>
    psl       (time, cell) float32 919GB dask.array<chunksize=(1, 3145728), meta=np.ndarray>
    ts        (time, cell) float32 919GB dask.array<chunksize=(1, 3145728), meta=np.ndarray>
    uas       (time, height, cell) float32 919GB dask.array<chunksize=(1, 1, 3145728), meta=np.ndarray>
    vas       (time, height, cell) float32 919GB dask.array<chunksize=(1, 1, 3145728), meta=np.ndarray>
Attributes: (12/31)
    Conventions:           CF-1.7 CMIP-6.2
    activity_id:           EERIE
    data_specs_version:    01.00.32
    forcing_index:         1
    initialization_index:  1
    license:               EERIE model data produced by MPI-M is licensed und...
    ...                    ...
    parent_activity_id:    EERIE
    sub_experiment_id:     none
    experiment:            coupled control with fixed 1950's forcing (HighRes...
    source:                ICON-ESM-ER (2023): \naerosol: none, prescribed MA...
    institution:           Max Planck Institute for Meteorology, Hamburg 2014...
    sub_experiment:        none

In [6]:
ds

<xarray.Dataset> Size: 8TB
Dimensions:             (ncells: 5242880, height: 1, height_2: 1, time: 73051)
Coordinates:
    cell_sea_land_mask  (ncells) float64 42MB dask.array<chunksize=(5242880,), meta=np.ndarray>
  * height              (height) float64 8B 10.0
  * height_2            (height_2) float64 8B 2.0
    lat                 (ncells) float64 42MB dask.array<chunksize=(5242880,), meta=np.ndarray>
    lon                 (ncells) float64 42MB dask.array<chunksize=(5242880,), meta=np.ndarray>
  * time                (time) datetime64[ns] 584kB 1991-01-01T05:59:59 ... 2...
Dimensions without coordinates: ncells
Data variables:
    hus2m               (time, height_2, ncells) float32 2TB dask.array<chunksize=(1, 1, 5242880), meta=np.ndarray>
    psl                 (time, ncells) float32 2TB dask.array<chunksize=(1, 5242880), meta=np.ndarray>
    ts                  (time, ncells) float32 2TB dask.array<chunksize=(1, 5242880), meta=np.ndarray>
    uas                 (time, height, ncells) float32 2TB dask.array<chunksize=(1, 1, 5242880), meta=np.ndarray>
    vas                 (time, height, ncells) float32 2TB dask.array<chunksize=(1, 1, 5242880), meta=np.ndarray>
Attributes: (12/31)
    Conventions:           CF-1.7 CMIP-6.2
    activity_id:           EERIE
    data_specs_version:    01.00.32
    forcing_index:         1
    initialization_index:  1
    license:               EERIE model data produced by MPI-M is licensed und...
    ...                    ...
    parent_activity_id:    EERIE
    sub_experiment_id:     none
    experiment:            coupled control with fixed 1950's forcing (HighRes...
    source:                ICON-ESM-ER (2023): \naerosol: none, prescribed MA...
    institution:           Max Planck Institute for Meteorology, Hamburg 2014...
    sub_experiment:        none

In [7]:
def magnitude(a, b):
    func = lambda x, y: np.sqrt(x**2 + y**2)
    return xr.apply_ufunc(func, a, b)

In [8]:
array = xr.DataArray([1, 2, 3], coords=[("x", [0.1, 0.2, 0.3])])
magnitude(array, -array)

<xarray.DataArray (x: 3)> Size: 24B
array([1.41421356, 2.82842712, 4.24264069])
Coordinates:
  * x        (x) float64 24B 0.1 0.2 0.3

In [9]:
magnitude(3, 4)
magnitude(3, np.array([0, 4]))
magnitude(array, 0)

<xarray.DataArray (x: 3)> Size: 24B
array([1., 2., 3.])
Coordinates:
  * x        (x) float64 24B 0.1 0.2 0.3

In [15]:
def mean(obj, dim):
    # note: apply always moves core dimensions to the end
    return xr.apply_ufunc(
        np.mean, obj, input_core_dims=[[dim]], kwargs={"axis": -1}
    )

In [16]:
array

<xarray.DataArray (x: 3)> Size: 24B
array([1, 2, 3])
Coordinates:
  * x        (x) float64 24B 0.1 0.2 0.3

In [17]:
mean(array, 'x')

<xarray.DataArray ()> Size: 8B
array(2.)

In [18]:
d = np.zeros((12, 324, 432))
da = xr.DataArray(d, dims=['time', 'lat', 'lon'])

In [28]:
def ll2hp(d):
    return np.zeros(12 * 4**5)

def wrapper(d):
    print(f"received x of type {type(d)}, shape {d.shape}")
    return ll2hp(d)

In [29]:
wrapper(da.values)

received x of type <class 'numpy.ndarray'>, shape (12, 324, 432)


array([0., 0., 0., ..., 0., 0., 0.])

In [38]:
xr.apply_ufunc(wrapper, da, input_core_dims=[['time']], output_core_dims=[['time']])

received x of type <class 'numpy.ndarray'>, shape (324, 432, 12)


ValueError: applied function returned data with an unexpected number of dimensions. Received 1 dimension(s) but expected 3 dimensions with names ('lat', 'lon', 'time'), from:

array([0., 0., 0., ..., 0., 0., 0.])

In [33]:
da.dims

('time', 'lat', 'lon')